In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
sql1 = textwrap.dedent("""
    IF OBJECT_ID('tempdb..#Intensity') IS NOT NULL
        DROP TABLE #Intensity
""")
sql2 = textwrap.dedent("""
    SELECT userid
          ,date
          ,COUNT(url) AS Intensity
    INTO #Intensity
    FROM [data].[dbo].[health_records]
    GROUP BY userid, date
    ORDER BY userid, date
""")
sql3 = textwrap.dedent("""
    SELECT Intensity
          ,COUNT(Intensity)
    FROM #Intensity
    GROUP BY Intensity
    ORDER BY Intensity
""")

In [ ]:
sql4 = textwrap.dedent("""
    SELECT userid
          ,date
          ,COUNT(url) AS Intensity
    FROM [data].[dbo].[health_records]
    GROUP BY userid, date
    ORDER BY userid, date
""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
cursor.execute(sql1)
cursor.execute(sql2)
rows=cursor.execute(sql3).fetchall()

In [ ]:
rowsx=cursor.execute(sql4).fetchall()

In [ ]:
df = pd.DataFrame( [[ij for ij in i] for i in rows] )
df.rename(columns={0:'Intensity', 1: 'Sessions'}, inplace=True)

In [ ]:
# 查看intensity的具体情况
dfx = pd.DataFrame( [[ij for ij in i] for i in rowsx] )
dfx.rename(columns={0:'userid', 1: 'date', 2:'intensity'}, inplace=True)
# 剔除intensity小于3的情况
a=list(dfx.intensity)
b=set([1,2])
ret=list(set(a)^b)
dfx = dfx[dfx.intensity.isin(ret)]
# intensity的描述统计
dfx.describe()

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
# intensity的箱线图
plt.figure()
p=dfx.boxplot(return_type='dict')
# 获取异常值
y=p['fliers'][0].get_ydata()
y.sort()
plt.show()
len(y)
y

In [ ]:
plotly.tools.set_credentials_file(username='tenaghirmb', api_key='please_use_your_own_key')

plotly.offline.init_notebook_mode(connected=True)

In [ ]:
trace = go.Scatter(
    x = df[df.Intensity.isin(range(3,242))]['Intensity'],
    y = df[df.Intensity.isin(range(3,242))]['Sessions'],
    mode = 'lines',
)

data = [trace]

layout = go.Layout(
    xaxis=dict(
        title='Intensity of Use',
        titlefont=dict(
            size=16,
        ),
        ticklen=2,
        tickfont=dict(
            size=12,
        ),
        nticks=8,
        #range=[3,241],
        #type='log',
    ),
    yaxis=dict(
        title='Number of Sessions',
        titlefont=dict(
            size=16,
        ),
        tickfont=dict(
            size=11,
        ),
        nticks=20,
    ),
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='Number of Visits each Session', image='png')

#py.iplot(fig, filename='Number of Visits each Session')